In [347]:
import selenium.webdriver as webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time
import re
from collections import defaultdict
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from io import BytesIO
from io import StringIO
import subprocess
import shutil
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
import openai
from openai import OpenAI
import os

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from deutschland.bundesanzeiger import Bundesanzeiger


def get_chromium_version() -> str:
    try:
        result = subprocess.run(['chromium', '--version'], capture_output=True, text=True)
        version = result.stdout.split()[1]
        return version
    except Exception as e:
        return str(e)

def get_chromedriver_version() -> str:
    try:
        result = subprocess.run(['chromedriver', '--version'], capture_output=True, text=True)
        version = result.stdout.split()[1]
        return version
    except Exception as e:
        return str(e)

def get_chromedriver_path() -> str:
    return shutil.which('chromedriver')

def get_webdriver_options(headless=False):
    options = Options()
    
    if headless:
        # The newer headless argument is "--headless=new" for Chrome 109+
        # If you experience compatibility issues, fallback to "--headless"
        options.add_argument("--headless=new")

    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-features=NetworkService")
    options.add_argument("--window-size=1920x1080")
    options.add_argument("--disable-features=VizDisplayCompositor")
    options.add_argument('--ignore-certificate-errors')
    return options

def get_webdriver_service():
    service = Service(
        executable_path=get_chromedriver_path()
    )
    return service

def run_selenium(domain, headless=False):
    """
    Attempts to fetch the HTML content of `domain` using Selenium.
    If headless=True, runs in headless mode; otherwise, opens a visible browser window.
    """
    html_content = None

    options = get_webdriver_options(headless=headless)
    service = get_webdriver_service()

    with webdriver.Chrome(options=options, service=service) as driver:
        try:
            driver.get(domain)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            html_content = driver.page_source
        except Exception as e:
            print(f"Error occurred while fetching {domain} in headless={headless} mode: {e}")

    return html_content

def extract_body_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    body_content = soup.body
    if body_content:
        return str(body_content)
    return ""

def clean_body_content(body_content):
    soup = BeautifulSoup(body_content,'html.parser')

    for script_or_style in soup(['script', 'style']):
        script_or_style.extract()

    cleaned_content = soup.get_text(separator='\n')
    cleaned_content = '\n'.join(
        line.strip() for line in cleaned_content.splitlines() if line.strip()
        )
    return cleaned_content

def split_dom_content(dom_content, max_length = 8000):
    return [
        dom_content[i : i + max_length] for i in range(0,len(dom_content), max_length)
    ]

def is_valid_url(url):
    parsed = urlparse(url)
    return all([parsed.scheme, parsed.netloc])  # Checks for scheme (http/https) and netloc (domain)


def choose_links(links):
    print("Here are the links to explore:\n")
    for i, link in enumerate(links):
        print(f"{i}: {link}")
    
    # Example input: "0,2"
    user_input = input("\nSelect the indexes of links you want to explore, separated by commas:\n> ")
    
    # Convert the input string into a list of integers
    indexes = [int(x.strip()) for x in user_input.split(",") if x.strip().isdigit()]
    
    chosen_links = [links[i] for i in indexes if i < len(links)]
    return chosen_links


def get_all_links(domain):
    links_to_visit = []
    try:
        html = run_selenium(domain)

        # If run_selenium returned None, bail out early:
        if html is None:
            print(f"Error: No HTML returned from run_selenium for '{domain}'")
            return links_to_visit
        
        # save content
        body_content = extract_body_content(html)  # Extract body content
        cleaned_content = clean_body_content(body_content)  # Clean the content
        
        #pull respective links
        soup = BeautifulSoup(html, 'html.parser')
        for a_tag in soup.find_all('a', href=True):
            # Resolve relative URLs
            full_url = urljoin(domain, a_tag['href'])
            links_to_visit.append(full_url)
        

    except Exception as e:
        # Refer to the domain (which is always defined), not full_url
        print(f"Error visiting domain '{domain}': {e}")

    return links_to_visit, cleaned_content

def filter_urls_by_stem(stem_url, url_list):
    """
    Compares each URL in url_list to stem_url.
    If a URL starts with the stem_url, it's added to a matches list.
    
    :param stem_url: A string representing the 'stem' portion of the URL.
    :param url_list: A list of full URLs to check against the stem URL.
    :return: A list of URLs that match the stem URL.
    """
    matches = []
    for random_url in url_list:
        # Simple check: does the random_url begin with stem_url?
        if random_url.startswith(stem_url) and random_url not in matches and random_url != stem_url:
            matches.append(random_url)
    return matches


from concurrent.futures import ThreadPoolExecutor, as_completed

def process_link(link, depth=5):
    """
    Processes a single link by:
      1) Getting its sub_links and sub_content
      2) Filtering links based on the stem
      3) Recursively expanding up to 'depth' levels
      4) Returning a tuple: (link, link_set, content_dict_sub)
    """
    # 1) Get sub_links and sub_content for this chosen link
    sub_links, sub_content = get_all_links(link)

    # Dictionary to store content for all pages under this link
    content_dict_sub = {}
    content_dict_sub[link] = sub_content

    # Maintain a set of all discovered links for this domain
    link_set = set()

    # 2) Filter sub_links based on the stem
    first_pass = filter_urls_by_stem(link, sub_links)
    link_set.update(first_pass)

    frontier = set(first_pass)

    # 3) Expand up to specified depth
    for depth_level in range(depth):
        next_frontier = set()
        for current_link in frontier:
            current_links, current_content = get_all_links(current_link)
            content_dict_sub[current_link] = current_content

            new_links = set(filter_urls_by_stem(current_link, current_links))
            new_links -= link_set

            link_set.update(new_links)
            next_frontier.update(new_links)

        frontier = next_frontier

    # 4) Return result: the original link, the aggregated link set, and all content
    return (link, link_set, content_dict_sub)

def preprocess(chosen_links, depth=5):
    start_time = time.time()  # Record the start time

    # We’ll store all the link expansions here
    link_library = {}
    # We’ll store the body/content here. Key = URL, Value = text content
    content_dict_master = {}

    # Use ThreadPoolExecutor for parallelism
    max_workers = min(8, len(chosen_links))  # Example: up to 8 threads or # of links
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit one process_link task per chosen link
        future_to_link = {
            executor.submit(process_link, link, depth): link
            for link in chosen_links
        }

        # Collect results as they complete
        for future in as_completed(future_to_link):
            link = future_to_link[future]
            try:
                processed_link, link_set, content_dict_sub = future.result()

                # Store the results in their respective data structures
                link_library[processed_link] = list(link_set)
                content_dict_master[processed_link] = content_dict_sub
            except Exception as e:
                print(f"Error processing link {link}: {e}")

    end_time = time.time()
    print(f"\nPreprocess finished in {end_time - start_time:.2f} seconds")

    return link_library, content_dict_master

def preprocess_content(content_dict):
    master_dict = {}
    for key in content_dict.keys():
        master_dict[key] = ''.join(content_dict[key].values())
    return master_dict

def run_chatgpt(prompt, model = "gpt-3.5-turbo"):
    client = OpenAI(
        api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    )
    response = client.chat.completions.create(
        model=model,
        messages=[
       {"role": "system", "content": "You are an advanced data extraction assistant."},
       {"role": "user", "content": prompt}
        ],
        temperature=0.0  # For deterministic responses
    )
    return response

def first_pass_with_chatGPT(dom_chunks, parse_description):
    """
    Sends chunks of website content to OpenAI's GPT API for parsing based on a description.
    """

    # Loop through chunks and get responses
    parsed_results = []
    for i, chunk in enumerate(dom_chunks, start=1):
        template = (
            f'You are tasked with extracting specific information from the following text content: {chunk}.'
            'Please follow these instructions carefully: \n\n'
            f' 1. ** Extract Information** : Only extract the information that directly matches the provided description {parse_description}'
            ' 2. ** No Extra Content** : Do not include any additional text, comments or explanations in your response.'
            ' 3. ** Empty Response** : If no information matches the description return to an empty string'
            ' 4. ** Direct Data** : Your response should contain only data that is explicitly requested, with no other text.'
            ' 5. ** Format** : **Single Table**: Output exactly one table, no extra lines or text.'
                            '  - Use semicolons (`;`) to separate columns.'
                            '  - Properly quote cells with ("") if they contain semicolons or commas.'
                            '  - If a cell has no data, use `empty`.'
            )
        prompt = template
        print(f"Processing chunk {i}/{len(dom_chunks)}...")

        try:
            response = run_chatgpt(prompt)
            parsed_results.append(response.choices[0].message.content)

        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
            parsed_results.append("")  # Append an empty string if there's an error

    initial_result = "\n".join(parsed_results)

    return initial_result

def parse_semicolon_csv(llm_output, delimiters = None):
    """
    Extracts table data from LLM output (potentially wrapped in ```csv code fences),
    dynamically handles multiple delimiters, and ensures the resulting DataFrame has multiple columns.
    
    :param llm_output: The text output from the LLM containing tabular data.
    :param delimiters: List of possible delimiters to handle, e.g., [",", ";", "\t", "|"].
                       Defaults to common delimiters.
    :return: A pandas DataFrame containing the parsed table data.
    """
    # Default delimiters if none are provided
    if delimiters is None:
        delimiters = [",", ";", "\t", "|"]
    
    # 1) Extract the CSV text from code fences, if present
    pattern = r"```csv\n(.*?)```"
    match = re.search(pattern, llm_output, flags=re.DOTALL | re.IGNORECASE)

    if match:
        csv_text = match.group(1).strip()
    else:
        # If no fenced block found, assume entire output is the CSV
        csv_text = llm_output.strip()

    # 2) Basic cleanup: remove trailing spaces or lines
    lines = [line.strip() for line in csv_text.splitlines() if line.strip()]
    csv_text = "\n".join(lines)

    # 3) Try parsing the CSV with each delimiter
    for delimiter in delimiters:
        try:
            # Attempt to parse the text into a DataFrame
            df = pd.read_csv(
                StringIO(csv_text),
                sep=delimiter,
                engine="python",
                quotechar='"',
                on_bad_lines="skip"  # Skip lines that don't match columns
            )
            # Check if the resulting DataFrame has more than one column
            if df.shape[1] > 1:
                break
        except Exception:
            # Ignore errors and try the next delimiter
            continue
    else:
        # If none of the delimiters work, raise an error
        raise ValueError(
            "Could not parse the table data with any of the specified delimiters or the result only has one column."
        )
    
    # 4) Remove empty and unnamed columns
    df.dropna(axis="columns", how="all", inplace=True)
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # 5) Clean column names
    df.columns = [col.strip('" ').strip() for col in df.columns]

    # 6) Reset the index
    df.reset_index(drop=True, inplace=True)

    return df

def scrape_to_df(content_dict,parse_description):
    start_time = time.time()
    outputs = {}
    master_dict = preprocess_content(content_dict)
    counter = 0
    for key in master_dict:
        counter += 1
        print(f'Processing {key}, {counter} out of {len(master_dict.keys())} datasets to process.')
        dom_content = master_dict[key]
        dom_chunks = split_dom_content(dom_content)
        csv_output = first_pass_with_chatGPT(dom_chunks,parse_description)
        outputs[key] = parse_semicolon_csv(csv_output)
    end_time = time.time()
    print(f"\nParsing scrape results finished in {end_time - start_time:.2f} seconds")
    return outputs

def run_research(website,parse_description):
    links,content = preprocess(website)
    output = scrape_to_df(content,parse_description)
    return output

def select_topics():
    def is_valid_url(url):
        pattern = re.compile(r'^(https?:\/\/)?([\w.-]+)+(:\d+)?(\/[\w._-]*)*\/?$')
        return re.match(pattern, url)

    website_input = input("\nPlease paste the websites you'd like to explore, separated by commas: ")
    if not website_input.strip():
        print("No websites entered. Please try again.")
        return []

    websites = [str(x.strip().lower().rstrip("/")) for x in website_input.split(",") if x.strip()]
    valid_websites = [x for x in websites if is_valid_url(x)]

    if len(valid_websites) == 0:
        print("No valid websites provided.")
        return []

    print(f"Websites selected: {', '.join(valid_websites)}")
    return valid_websites

def user_selects(players = None):
    if players is None:
        domains = select_topics()
    else:
        domains = players.tolist() if isinstance(players, pd.Series) else players
    domain_links = {}
    for domain in domains:
        initial_links, initial_content = get_all_links(domain)
        domain_links[domain] = choose_links(initial_links)
    return domain_links

def user_task():
    user_input = input('\nPlease let DumbResearch know:\n1. What web content it is looking at \n2. What columns you would like your research output table to have \n\nSee example below: \n"These are the solutions offered by a german cyber security player (input is provided in German), please provide a table or tables that list the respective products / solutions with important information on the respective products e.g. their descriptions, features, target customers, KPIs"\n')
    return user_input

def initialize_dumbresearch(players = None):
    if players is not None and not isinstance(players, (list, pd.Series)):
        raise ValueError("`players` must be a list or Pandas Series of URLs")
    domains_selected = user_selects(players)
    parse_description = user_task()
    return domains_selected, parse_description

def dumbresearch_workflow(players = None):
    domains, parse_description = initialize_dumbresearch(players)
    print(domains)
    outputs = {}
    for key in domains.keys():
        links, content = preprocess(domains[key])
        output = scrape_to_df(content, parse_description)
        outputs[key] = output
    return outputs

def find_competitors():
    company = input('\n**Target**\nPlease let DumbResearch know, on which company you are conducting desktop research')
    company_context = input('\n**Target description**\nPlease let DumbResearch know more about the company, i.e. what industry, geographies and if you want it to pay particular attention to anything or exclude something in its research!\n')
    result = parse_semicolon_csv(competitors_with_chatGPT(company,company_context))
    return result


def competitors_with_chatGPT(company, company_context):
    """
    Sends chunks of website content to OpenAI's GPT API for parsing based on a description.
    """

    # Loop through chunks and get responses
    parsed_results = []
    template = (
        f'You are tasked with finding the competitors of a given company called {company}, with the below context provided: {company_context}.'
        'Please follow these instructions carefully: \n\n'
        ' 1. ** Extract Information** : Only extract the information that directly matches the provided description above'
        ' 2. ** Volume** : Find at least 10 competitors for each requests.'
        ' 3. ** Empty Response** : If no information matches the description return to an empty string'
        ' 4. ** Direct Data** : Your response should contain only data that is explicitly requested, with no other text.'
        ' 5. ** Format** : **Single Table**: Output exactly one table, no extra lines or text.'
                        '  - Use semicolons (`;`) to separate columns.'
                        '  - Properly quote cells with ("") if they contain semicolons or commas.'
                        '  - If a cell has no data, use `empty`.'
                        '  - Columns are the 1) respective competitors ("Competitor") 2) the industry they are in ("Industry") 3) the link to their website ("Website") 4) The name of the legal entity that is registered with the authorities ("Legal entity")'
        )
    prompt = template
    print(f"Finding competitors of {company}...")

    try:
        response = run_chatgpt(prompt, model = 'gpt-4o')
        parsed_results.append(response.choices[0].message.content)

    except Exception as e:
        print(f"Error processing chunk {i}: {e}")
        parsed_results.append("")  # Append an empty string if there's an error

    initial_result = "\n".join(parsed_results)

    return initial_result

def competitor_search_workflow():
    result = find_competitors()
    
    # Display found competitors
    for i, link in enumerate(result['Website']):
        print(f"{i}: {link}")
        
    # Ask the user for additional competitors
    user_input = input(
        "\nThese are the competitors DumbResearch found. "
        "Do you have any additional players you would like to add to the search? "
        "Please add their respective websites, separated by a comma:\n"
    )
    add_players = [str(x.strip().lower().rstrip("/")) for x in user_input.split(",") if x.strip()]
    
    # Add new competitors to the result DataFrame
    for i in add_players:
        new_row = {'Competitor': None, 'Industry': None, 'Website': i, 'Legal entity': None}
        result = pd.concat([result, pd.DataFrame([new_row])], ignore_index=True)
    
    # Display the updated list of competitors
    for i, link in enumerate(result['Website']):
        print(f"{i}: {link}")
    
    # Ask the user to select competitors to keep
    user_input = input(
        "\nThis is the current set of competitors. "
        "Select the indexes of competitors you want to keep, separated by commas:\n> "
    )
    
    try:
        # Convert the input string into a list of integers
        indexes = [int(x.strip()) for x in user_input.split(",") if x.strip().isdigit()]
    except ValueError:
        print("Invalid input. Please enter a comma-separated list of numbers.")
        return {}
    
    # Validate indexes and filter the selected competitors
    chosen_competitors = result.iloc[indexes].reset_index(drop=True) if indexes else pd.DataFrame()
    
    return chosen_competitors

def search_research_workflow():
    competitor_set = competitor_search_workflow()
    print(f"Now looking at the websites of your competitor set {competitor_set['Website']}")
    output = dumbresearch_workflow(competitor_set['Website'])
    return output



import datetime
import pandas as pd

def parse_semicolon_csv_to_dataframe(csv_text):
    """
    Dynamically determine the max number of columns across all lines
    and use that to build the DataFrame.
    """
    lines = [line.strip() for line in csv_text.strip().splitlines() if line.strip()]
    if not lines:
        return pd.DataFrame()

    # Split everything first
    splitted_lines = [row.split(";") for row in lines]

    # Find the maximum column count
    max_cols = max(len(sl) for sl in splitted_lines)

    # First row is still used as headers, but it might not have all columns
    headers = splitted_lines[0]
    if len(headers) < max_cols:
        # Pad the header row with "ColumnX" or "empty" for missing columns
        missing_count = max_cols - len(headers)
        headers += [f"ExtraCol{i+1}" for i in range(missing_count)]
    elif len(headers) > max_cols:
        # If the header has more columns than any data row, just truncate it
        headers = headers[:max_cols]

    # For all subsequent rows, pad or truncate
    data_rows = []
    for row in splitted_lines[1:]:
        if len(row) < max_cols:
            row += ["empty"] * (max_cols - len(row))
        elif len(row) > max_cols:
            row = row[:max_cols]
        data_rows.append(row)

    df = pd.DataFrame(data_rows, columns=headers)
    return df


#############################################
# 2) Individual Extraction Functions
#############################################

def extract_income_statement(report_text):
    """
    Extracts the Income Statement from the given `report_text`.
    Returns a DataFrame, or empty DataFrame if none found.
    """
    # Single-task instructions to ChatGPT
    income_statement_instructions = """
    You are tasked with extracting only the Income Statement data
    from the following report. The output should be formatted as
    a semicolon-delimited table with a header row.

    Instructions:
    1. Only output the Income Statement (no other tables).
    2. Use semicolons (;) to separate columns.
    3. If certain cells are empty, write 'empty'.
    4. Do not include extra commentary or text outside the table.
    5. The first line should be the header row.

    Here is the input:
    {report_text}
    """

    prompt = income_statement_instructions.format(report_text=report_text)
    response_text = run_chatgpt(prompt)
    response_text = response_text.choices[0].message.content
    df = parse_semicolon_csv_to_dataframe(response_text)
    return df


def extract_operational_kpis(report_text):
    """
    Extracts important operational KPIs from the report text,
    such as production volume, customer count, churn rate, ARPU, etc.
    Returns a DataFrame.
    """
    kpi_instructions = """
    You are tasked with extracting only the operational KPIs from
    the following report. Examples of such KPIs include:
    - Number of customers or clients
    - Churn rate
    - ARPU (Average Revenue per User)
    - Employee headcount
    - Production volume
    - Utilization rates
    - Other similar quantitative metrics

    Output a semicolon-delimited table with a header row.
    Each row should contain:
       - KPI Name
       - KPI Value
       - (Optionally) a KPI Description

    Only output this single table, no other text.

    Here is the input:
    {report_text}
    """

    prompt = kpi_instructions.format(report_text=report_text)
    response_text = run_chatgpt(prompt)
    response_text = response_text.choices[0].message.content
    df = parse_semicolon_csv_to_dataframe(response_text)
    return df


def extract_qualitative_info(report_text):
    """
    Extracts only qualitative information, e.g.,
    management commentary, forward-looking statements, strategic objectives, etc.
    Returns a DataFrame with two columns: label, content.
    """
    qualitative_instructions = """
    You are tasked with extracting only qualitative information
    such as management commentary, forward-looking statements,
    strategic objectives, or other narrative insights from the
    following report. Each piece of qualitative info should be
    categorized under a 'label', with the actual commentary in 'content'.

    Output a semicolon-delimited table with this header:
    "label";"content"

    Only output this single table, no extra text or lines.

    Here is the input:
    {report_text}
    """

    prompt = qualitative_instructions.format(report_text=report_text)
    response_text = run_chatgpt(prompt)
    response_text = response_text.choices[0].message.content
    df = parse_semicolon_csv_to_dataframe(response_text)
    return df


#############################################
# 3) Orchestrator for a Single Report
#############################################

def parse_single_report(report_data):
    """
    Given one report (dictionary with keys like date, company, report),
    runs the three extraction functions and returns a dictionary of DataFrames.
    e.g. {
        "Income Statement": <DataFrame>,
        "KPIs": <DataFrame>,
        "Qualitative": <DataFrame>
    }
    """

    report_text = report_data.get("report", "")
    if not report_text:
        # Return empty placeholders if no text
        return {
            "Income Statement": pd.DataFrame(),
            "KPIs": pd.DataFrame(),
            "Qualitative": pd.DataFrame()
        }

    # Calls each extraction function separately
    income_df = extract_income_statement(report_text)
    kpi_df = extract_operational_kpis(report_text)
    qual_df = extract_qualitative_info(report_text)

    return {
        "Income Statement": income_df,
        "KPIs": kpi_df,
        "Qualitative": qual_df
    }


#############################################
# 4) parse_bundesanzeiger_reports
#############################################

def parse_bundesanzeiger_reports(reports_dict):
    """
    Takes a dictionary of reports, keyed by unique IDs, where each value 
    is a dictionary containing 'date', 'company', 'report' text, etc.

    Returns a final structure:
    {
        <company_name>: {
            <year>: {
                <report_id>: {
                   "Income Statement": DataFrame,
                   "KPIs": DataFrame,
                   "Qualitative": DataFrame
                },
                ...
            },
            ...
        },
        ...
    }
    """

    master_outputs = {}

    for report_id, report_data in reports_dict.items():
        # Identify the company
        company_name = report_data.get("company", "Unknown Company")

        # Parse out the year from the date
        raw_date = report_data.get("date", None)
        parsed_year = None
        if isinstance(raw_date, datetime.date):
            parsed_year = raw_date.year
        elif isinstance(raw_date, str):
            try:
                dt = datetime.datetime.strptime(raw_date, "%Y-%m-%d")
                parsed_year = dt.year
            except ValueError:
                print(f"Could not parse date for report {report_id}: {raw_date}")
        if not parsed_year:
            print(f"Skipping report {report_id}: No valid year.")
            continue

        # Build structure: master_outputs[company][year][report_id]
        if company_name not in master_outputs:
            master_outputs[company_name] = {}
        if parsed_year not in master_outputs[company_name]:
            master_outputs[company_name][parsed_year] = {}

        # Parse the single report using the orchestrator function
        results_for_this_report = parse_single_report(report_data)

        # Save the results
        master_outputs[company_name][parsed_year][report_id] = results_for_this_report

    return master_outputs

In [290]:
result = preprocess(['https://www.stackinfra.com/solutions/'])


Preprocess finished in 8.70 seconds


In [402]:
reports = parse_bundesanzeiger_reports(data)

2024-12-29 19:21:39.787 INFO    httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-29 19:21:41.073 INFO    httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-29 19:21:46.391 INFO    httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-29 19:21:48.798 INFO    httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-29 19:21:50.081 INFO    httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-29 19:21:54.122 INFO    httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-29 19:21:59.794 INFO    httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-29 19:22:00.352 INFO    httpx: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-29 19:22:04.821 INFO    httpx: HTTP Requ

In [404]:
reports['Myra Security GmbH'].keys()

dict_keys([2023, 2021, 2020, 2019, 2017, 2016, 2015, 2014])

In [450]:
import requests

api_key = os.getenv("COMPANIES_HOUSE_API_KEY")

import requests

def get_company_number(company_name, api_key):
    """
    Fetches the company number of the first search result for a given company name.

    Args:
        company_name (str): The name of the company to search for.
        api_key (str): Your Companies House API key.

    Returns:
        str: The company number of the first search result.
        None: If no company is found or an error occurs.
    """
    base_url = "https://api.company-information.service.gov.uk"
    search_url = f"{base_url}/search/companies?q={company_name}"

    # Call the search endpoint
    response = requests.get(search_url, auth=(api_key, ''))

    if response.status_code != 200:
        print(f"Error: Search request failed with status code {response.status_code}")
        return None

    data = response.json()

    # Check if the search returned any results
    if "items" not in data or len(data["items"]) == 0:
        print(f"No results found for company name: {company_name}")
        return None

    # Extract the company number of the first result
    company_number = data["items"][0]["company_number"]
    return company_number

def get_filing_history(company_number, api_key):
    """
    Fetches the complete filing history of a company using its company number,
    handling pagination to collect all items.

    Args:
        company_number (str): The unique company number of the company.
        api_key (str): Your Companies House API key.

    Returns:
        list: A complete list of filing history items.
        None: If no filing history is found or an error occurs.
    """
    base_url = "https://api.company-information.service.gov.uk"
    filing_history_url = f"{base_url}/company/{company_number}/filing-history"

    all_items = []  # List to store all filing history items
    start_index = 0  # Initial page index
    items_per_page = 25  # Default items per page (as per Companies House API documentation)

    while True:
        # Add pagination parameters to the request
        params = {
            "start_index": start_index,
            "items_per_page": items_per_page
        }

        # Call the filing history endpoint
        response = requests.get(filing_history_url, params=params, auth=(api_key, ''))

        if response.status_code != 200:
            print(f"Error: Filing history request failed with status code {response.status_code}")
            return None

        data = response.json()

        # Append the items from the current page to the list
        items = data.get("items", [])
        all_items.extend(items)

        # Check if we have fetched all items
        total_count = data.get("total_count", 0)
        if len(all_items) >= total_count:
            break

        # Update the start_index for the next page
        start_index += items_per_page

    return all_items
    
def get_filing_history(company_number, api_key):
    """
    Fetches the complete filing history of a company using its company number,
    handling pagination to collect all items.

    Args:
        company_number (str): The unique company number of the company.
        api_key (str): Your Companies House API key.

    Returns:
        list: A complete list of filing history items.
        None: If no filing history is found or an error occurs.
    """
    base_url = "https://api.company-information.service.gov.uk"
    filing_history_url = f"{base_url}/company/{company_number}/filing-history"

    all_items = []  # List to store all filing history items
    start_index = 0  # Initial page index
    items_per_page = 25  # Default items per page (as per Companies House API documentation)

    while True:
        # Add pagination parameters to the request
        params = {
            "start_index": start_index,
            "items_per_page": items_per_page
        }

        # Call the filing history endpoint
        response = requests.get(filing_history_url, params=params, auth=(api_key, ''))

        if response.status_code != 200:
            print(f"Error: Filing history request failed with status code {response.status_code}")
            return None

        data = response.json()

        # Append the items from the current page to the list
        items = data.get("items", [])
        all_items.extend(items)

        # Check if we have fetched all items
        total_count = data.get("total_count", 0)
        if len(all_items) >= total_count:
            break

        # Update the start_index for the next page
        start_index += items_per_page

    return all_items


def get_company_filing_history(company_name, api_key):
    """
    Fetches the filing history of a company using its name.

    Args:
        company_name (str): The name of the company to search for.
        api_key (str): Your Companies House API key.

    Returns:
        dict: Filing history details or an error message.
    """
    # Step 1: Get the company number
    company_number = get_company_number(company_name, api_key)

    if not company_number:
        return {"error": f"Could not find company number for: {company_name}"}

    # Step 2: Get the filing history
    filing_history = get_filing_history(company_number, api_key)

    if not filing_history:
        return {"error": f"Could not retrieve filing history for company number: {company_number}"}

    return {
        "company_name": company_name,
        "company_number": company_number,
        "filing_history": filing_history,
    }

import requests

import os

def fetch_document(document_id, api_key, output_path):
    """
    Fetches a document from the Companies House Document API and saves it locally.

    Args:
        document_id (str): The ID of the document to fetch.
        api_key (str): Your Companies House API key.
        output_path (str): The file path (or directory) where the document will be saved.

    Returns:
        bool: True if the document was successfully fetched and saved, False otherwise.
    """
    # If the output path is a directory, append a default filename
    if os.path.isdir(output_path):
        output_path = os.path.join(output_path, f"{document_id}.pdf")

    base_url = "https://document-api.company-information.service.gov.uk"
    document_url = f"{base_url}/document/{document_id}/content"

    # Headers for the request
    headers = {
        "Accept": "application/pdf",  # Request the document as a PDF
    }

    # Send the request to fetch the document
    response = requests.get(document_url, headers=headers, auth=(api_key, ''), allow_redirects=False)

    if response.status_code == 302:  # The document is available
        # Follow the redirect URL in the 'Location' header to download the document
        redirect_url = response.headers.get("Location")
        if not redirect_url:
            print(f"Error: Redirect URL not provided for document_id {document_id}.")
            return False

        # Fetch the document content
        document_response = requests.get(redirect_url)

        if document_response.status_code == 200:
            # Save the document locally
            with open(output_path, "wb") as file:
                file.write(document_response.content)
            print(f"Document {document_id} saved to {output_path}.")
            return True
        else:
            print(f"Error: Failed to fetch document content. Status code {document_response.status_code}.")
            return False

    elif response.status_code == 401:
        print("Error: Unauthorized. Check your API key.")
        return False
    else:
        print(f"Error: Failed to fetch document. Status code {response.status_code}.")
        return False

In [441]:
company_name = 'G.Network Communications Ltd'

os.environ["COMPANIES_HOUSE_API_KEY"] = "cd88de94-9a32-496c-8970-bdaa3ec3d61c"

api_key = os.getenv("COMPANIES_HOUSE_API_KEY")

result = get_company_filing_history(company_name,api_key)


In [451]:
path = '/Users/roscoe/Downloads'
doc_id = 'dXXqzTVwa0UWdU-WTcDHaSuW0wIxOZc4flgHPf4isdk'
fetch_document(doc_id, api_key, path)

Document dXXqzTVwa0UWdU-WTcDHaSuW0wIxOZc4flgHPf4isdk saved to /Users/roscoe/Downloads/dXXqzTVwa0UWdU-WTcDHaSuW0wIxOZc4flgHPf4isdk.pdf.


True

In [446]:
for report in result['filing_history']:
    if report['description'] == 'accounts-with-accounts-type-group':
        print(report)


{'transaction_id': 'MzQzMTUxNTg3OWFkaXF6a2N4', 'barcode': 'AD8PZ9A8', 'type': 'AA', 'date': '2024-08-13', 'category': 'accounts', 'description': 'accounts-with-accounts-type-group', 'description_values': {'made_up_date': '2024-03-31'}, 'pages': 65, 'action_date': '2024-03-31', 'paper_filed': True, 'links': {'self': '/company/10057745/filing-history/MzQzMTUxNTg3OWFkaXF6a2N4', 'document_metadata': 'https://document-api.company-information.service.gov.uk/document/dXXqzTVwa0UWdU-WTcDHaSuW0wIxOZc4flgHPf4isdk'}}
{'transaction_id': 'MzQwNDYxMTMwMmFkaXF6a2N4', 'barcode': 'BCIEMDJM', 'type': 'AA', 'date': '2023-12-28', 'category': 'accounts', 'description': 'accounts-with-accounts-type-group', 'description_values': {'made_up_date': '2023-03-31'}, 'pages': 58, 'action_date': '2023-03-31', 'paper_filed': True, 'links': {'self': '/company/10057745/filing-history/MzQwNDYxMTMwMmFkaXF6a2N4', 'document_metadata': 'https://document-api.company-information.service.gov.uk/document/mGTBsga6S0Z8l3XM8NegVKa

In [341]:
import re

def parse_full_report(data):
    """
    Parse a financial report, preserving both quantitative and qualitative information.
    
    Args:
    - data (dict): A dictionary containing 'date', 'name', 'company', 'report', and other metadata.
    
    Returns:
    - dict: A structured dictionary with parsed financial and qualitative information.
    """
    report = data.get("report", "")
    
    # Extract specific sections using delimiters
    aktiva_match = re.search(r"AKTIVA(.*?)PASSIVA", report, re.DOTALL)
    passiva_match = re.search(r"PASSIVA(.*?)Allgemeine Angaben", report, re.DOTALL)
    general_info_match = re.search(r"Allgemeine Angaben(.*?)Angaben zur Bilanz", report, re.DOTALL)
    bilanzi_info_match = re.search(r"Angaben zur Bilanz(.*?)Haftungsverhältnisse", report, re.DOTALL)
    employee_program_match = re.search(r"Virtuelles Mitarbeiterprogramm(.*?)Sonstige finanzielle Verpflichtungen", report, re.DOTALL)
    obligations_match = re.search(r"Sonstige finanzielle Verpflichtungen(.*?)München,", report, re.DOTALL)
    
    aktiva_data = aktiva_match.group(1).strip() if aktiva_match else ""
    passiva_data = passiva_match.group(1).strip() if passiva_match else ""
    general_info = general_info_match.group(1).strip() if general_info_match else ""
    bilanzi_info = bilanzi_info_match.group(1).strip() if bilanzi_info_match else ""
    employee_program = employee_program_match.group(1).strip() if employee_program_match else ""
    obligations = obligations_match.group(1).strip() if obligations_match else ""
    
    def parse_section(section):
        """
        Parse a financial section (e.g., AKTIVA or PASSIVA) into a structured list of items.
        """
        items = []
        lines = section.split("\n")
        
        for line in lines:
            match = re.match(r"(.*?)(\d{1,3}(?:\.\d{3})*,\d{2})(?:\s+(\d{1,3}(?:\.\d{3})*,\d{2}))?", line.strip())
            if match:
                item = match.group(1).strip()
                current_year = match.group(2).replace(".", "").replace(",", ".")
                previous_year = match.group(3).replace(".", "").replace(",", ".") if match.group(3) else None
                items.append({
                    "Item": item,
                    "Current Year (EUR)": float(current_year),
                    "Previous Year (EUR)": float(previous_year) if previous_year else None
                })
        return items
    
    # Parse AKTIVA and PASSIVA sections
    aktiva = parse_section(aktiva_data)
    passiva = parse_section(passiva_data)
    
    # Build the final structured output
    parsed_data = {
        "Metadata": {
            "Date": data.get("date"),
            "Company": data.get("company"),
            "Report Name": data.get("name"),
        },
        "Financial Data": {
            "AKTIVA": aktiva,
            "PASSIVA": passiva,
        },
        "Qualitative Information": {
            "General Info": general_info,
            "Balance Sheet Notes": bilanzi_info,
            "Employee Program": employee_program,
            "Financial Obligations": obligations,
        }
    }
    
    return parsed_data

In [346]:
data['14da79142f91a0e7bb80747555befe6d']

{'date': datetime.datetime(2023, 7, 3, 0, 0),
 'name': 'Jahresabschluss zum Geschäftsjahr vom 01.01.2021 bis zum 31.12.2021',
 'company': 'Myra Security GmbH',
 'report': '\n\n\n\n\xa0\n\n\n\n\n\n\n\nMyra Security GmbH\nMünchen\nJahresabschluss zum Geschäftsjahr vom 01.01.2021 bis zum 31.12.2021\nBILANZ zum 31. Dezember 2021\n\nAKTIVA \n\n\n\n\n\n\n\n\n\n\n\n\nGeschäftsjahr\nVorjahr\n\n\n\n\nEUR\nEUR\nEUR\n\n\n\n\nA. Anlagevermögen\n\n\n\n\n\n\n\n\nI. Immaterielle Vermögensgegenstände\n399.321,07\n\n\n229.063,00\n\n\nII. Sachanlagen \n935.797,80\n\n\n709.159,05\n\n\n\n\n\n\n1.335.118,87\n938.222,05\n\n\nB. Umlaufvermögen\n\n\n\n\n\n\n\n\nI. Vorräte\n51.625,54\n\n\n0,00\n\n\nII. Forderungen und sonstige Vermögensgegenstände\n886.016,79\n\n\n557.204,85\n\n\nIII. Kassenbestand,\n                     Bundesbankguthaben, Guthaben bei\n                     Kreditinstituten und Schecks\n2.285.848,38\n\n\n4.994.256,82\n\n\n\n\n\n\n3.223.490,71\n5.551.461,67\n\n\nC. Rechnungsabgrenzungsposten\n

In [270]:
content = run_selenium('https://www.link11.com/en/solutions/web-protection/web-ddos-protection/')
clean_content = clean_body_content(content)

In [271]:
clean_content

"Web DDoS Protection - Link11\nSolutions\nNetwork Security\nInfrastructure DDoS Protection\nCloud Insights\nWeb Protection\nWeb DDoS Protection\nBot Management\nZero Touch WAF\nAPI Protection\nWeb Performance\nSecure CDN\nSecure DNS\nIndustries\nHosting\nLogistic\nBanking & Finance\nE-Commerce\nUtilities\nOnline Gaming\nMedia\nPublic Sector\nBenchmark\nCustomers\nResources\nBlog\nGlossary\nPress\nDownloads\nVideos & Podcasts\nEvents\nAbout Link11\nCompany\nPatented DDoS Protection\nPartner\nCareer\nCarbon Neutrality\nContact\nen\nde\nLogin\nUnder attack?\nContact\nWeb DDoS Protection\nCloud-based DDoS protection for web applications\nAutomation ensures 24/7 protection\nZero time to mitigate for known,\n<\n10 seconds for new vectors\n4.3/5 Google Reviews\nBook a demo\nLearn more\nPrecise detection\nNo need to worry about complex attacks anymore: The system detects traffic anomalies and protects you from emerging threats in real time.\nFull automation\nNo work for you: manual operation i

In [229]:
initialize_dumbresearch()

KeyboardInterrupt: Interrupted by user

# User pastes in links they want to look at. Programm takes links and asks 
* which part of the website the user wants to look at
* for each of the subsection what they want the output to look like and a description
* where they want the output to be saved

# Program runs for each website, creates the tabular output for each of the websites subsections (e.g. product, locations)

# Tabular output is saved to the specified location

Appendix

parse_description = ('These are the solutions offered by a german cyber security player (input is provided in German), please provide a table or tables that list the respective products / solutions with important information on the respective products e.g. their descriptions, features, target customers, KPIs')

#Function to extract deeper-level headlines
def extract_headlines_with_depth(links, depth):
    headlines = set()
    for link in links:
        try:
            html = run_selenium(domain)
            time.sleep(10)

            soup = BeautifulSoup(html, 'html.parser')
            
            for a_tag in soup.find_all('a', href=True):
                # Resolve relative URLs
                full_url = urljoin(link, a_tag['href'])
                parsed = urlparse(full_url)
                path_parts = parsed.path.strip('/').split('/')
                if len(path_parts) == depth:
                    headlines.append(full_url)

        except Exception as e:
            print(f"Error visiting {full_url}: {e}")
            parsed = urlparse(link)
            path_parts = parsed.path.strip('/').split('/')
            if len(path_parts) == depth:
                headline = "/".join(path_parts[:depth])
                headlines.add(headline)
    return sorted(headlines)


def extract_second_level_headlines(links):
    # Parse URLs and extract paths
    headlines = set()  # Use a set to avoid duplicates
    for link in links:
        parsed = urlparse(link)
        path_parts = parsed.path.strip('/').split('/')

        # Match second-level paths (e.g., /solutions/, /locations/)
        headline = "/".join(path_parts[:1])
        headlines.add(headline)

    return sorted(headlines)
    
def get_select_data(headlines, domain,depth = 3):
    domain = domain
    subpage_links = set(get_all_links(domain))  # Assuming this fetches all sub-links from the domain
    
    # Dictionary to store concatenated text for each key
    content_dict = defaultdict(str)

    # Fetch and process links
    urls = {}
    for page in headlines:
        # Match URLs for the current page
        urls[page] = re_match_second_level_headlines(page, domain, subpage_links)
        for url in urls[page]:  # Iterate over the URLs for the current page
            if not is_valid_url(url):  # Validate URL before scraping
                print(f"Invalid URL skipped: {url}")
                continue

            print(f"Scraping URL: {url}")
            result = run_selenium(url)  # Scrape website content
            body_content = extract_body_content(result)  # Extract body content
            cleaned_content = clean_body_content(body_content)  # Clean the content

            # Concatenate the cleaned content to the respective key in the dictionary
            content_dict[page] += cleaned_content + " "  # Add a space between concatenated content

    return content_dict
    
    

def re_match_second_level_headlines(headline, domain, links_to_visit):
    matching_urls = []
    headline_url = urljoin(domain, headline)
    matching_urls.extend([url for url in links_to_visit if url.startswith(headline_url)])
    return matching_urls
    
    
        refinement_prompt = (f'This is the output from an LLM that has been fed multiple chunks of information: {initial_result}.'
                     'Please clean this output up by following these instructions carefully: \n\n'
                     '1. Format: The output will likely be in multiple tables, please condense all output into one table. IMPORTANT: “Output the table as valid CSV with exactly the same number of columns for all rows (have fill value "empty" if there is not input for a given cell), no extra lines, and properly quoted cells if they contain commas. Use a semicolon to seperate cells instead of a comma”'
                     '2. No duplication: The table you create should not contain duplicate rows, there should only be one row with relevant information for each e.g. product, location, customer etc'
                     '3. If the tables you receive have multiple different categories (e.g. solutions, locations, customers) still include them into one table and add a columns called "type" to your table that captures this information'
                     )






def parse_with_chatgpt(dom_chunks, parse_description):
    """
    Sends chunks of website content to OpenAI's GPT API for parsing based on a description.
    """

    template = (
        f'You are tasked with extracting specific information from the following text content: {dom_chunks}.'
        'Please follow these instructions carefully: \n\n'
        f' 1. ** Extract Information** : Only extract the information that directly matches the provided description {parse_description}'
        ' 2. ** No Extra Content** : Do not include any additional text, comments or explanations in your response.'
        ' 3. ** Empty Response** : If no information matches the description return to an empty string'
        ' 4. ** Direct Data** : Your response should contain only data that is explicitly requested, with no other text.'
        ' 5. ** Format** : **Single Table**: Output exactly one table, no extra lines or text.'
                        '  - Use semicolons (`;`) to separate columns.'
                        '  - Properly quote cells with ("") if they contain semicolons or commas.'
                        '  - If a cell has no data, use `empty`.'
        )

    # Loop through chunks and get responses
    parsed_results = []
    for i, chunk in enumerate(dom_chunks, start=1):
        prompt = templa
te.format(dom_chunks=chunk, parse_description=parse_description)
        print(f"Processing chunk {i}/{len(dom_chunks)}...")

        try:
            response = run_chatgpt(prompt)
            
            if isinstance(response, list):
                parsed_results.append(response[0].choices[0].message.content)
            else:
                parsed_results.append(response.choices[0].message.content)

            # Extract the content of the assistant's reply
            parsed_results.append(response[0].choices[0].message.content)

        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
            parsed_results.append("")  # Append an empty string if there's an error

    final_result = []

    initial_result = "\n".join(parsed_results)

    refinement_prompt = (f'This is the output from an LLM that has been fed multiple chunks of information: {initial_result}. Please merge them into a single CSV table with these requirements:'
                        '1. **Single Table**: Output exactly one table, no extra lines or text.'
                        '2. **Format**:'
                        '   - Use semicolons (`;`) to separate columns.'
                        '   - Properly quote cells with ("") if they contain semicolons or commas.'
                        '   - If a cell has no data, use `empty`.'
                        '3. **Columns**:'

                        '   - All rows must have the same number of columns.'
                        '   - No duplicate rows.'
                        '   - If the data covers multiple categories (e.g. solutions, locations, customers), add a column named `"Type"` to capture the category.'
                        '4. **No Duplicates**: Merge repeated rows or near-identical information into a single row.'
                        '5. **Final Output**:'
                        '   - No code fences or markdown.'
                        '   - Strictly the CSV text (with one header row).')

    if len(initial_result) > 12000:
        dom_chunks = split_dom_content(initial_result)
        for i, chunk in enumerate(dom_chunks,start=1):
            prompt_02 = refinement_prompt.format(initial_result=chunk)
            print(f"Processing refinement chunk {i}/{len(dom_chunks)}...")
            try:
                response = run_chatgpt(prompt_02)
                if isinstance(response, list):
                    final_result.append(response[0].choices[0].message.content)
                else:
                    final_result.append(response.choices[0].message.content)

                # Extract the content of the assistant's reply
                final_result.append(response[0].choices[0].message.content)

            except Exception as e:
                print(f"Error processing refinement chunk {i}: {e}")
                final_result.append("")  # Append an empty string if there's an error

    else:

        prompt_02 = refinement_prompt.format(initial_result=initial_result)
        print(f"Refining initial result...")
        try:
            response = run_chatgpt(prompt_02)
            if isinstance(response, list):
                final_result.append(response[0].choices[0].message.content)
            else:
                final_result.append(response.choices[0].message.content)

            # Extract the content of the assistant's reply
            final_result.append(response[0].choices[0].message.content)

        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
            final_result.append("")  # Append an empty string if there's an error

    return "\n".join(final_result)
                     